In [1]:
!pip3 install -U adapter-transformers
!pip3 install datasets

In [2]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("imdb")
dataset.num_rows

Reusing dataset imdb (C:\Users\Junfe\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

{'train': 25000, 'test': 25000, 'unsupervised': 50000}

In [3]:
train_dataset, valid_dataset= dataset["train"].train_test_split(test_size=0.2).values()
dataset = DatasetDict({"train":train_dataset,"test":dataset["test"], "validation":valid_dataset})
dataset.num_rows

Loading cached split indices for dataset at C:\Users\Junfe\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-21f5fdd3723ee985.arrow and C:\Users\Junfe\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-5ac3cbe7f17fba34.arrow


{'train': 20000, 'test': 25000, 'validation': 5000}

In [4]:
dataset["train"][0]

{'text': 'All the elements for a bad night at the movies are in place: dialog riddled with biological techno-babble, chintzy sets, balsa-wood acting, a horrific late-\'80s Casio score, and an overall look that suggests anything on the Sci-Fi Channel\'s programming schedule, circa 1993. Though "Metamorphosis" starts off with a lot of promise, the film unravels into bland idiocy and MST3K-style cheese as Clark Kent wannabe \'Doctor\' Peter Houseman (Gene LeBrock) is pressured into releasing information on his secretive projects. But when he tests his vague experiment on himself, he transforms into a vaguely-defined creature (that bears more than a passing resemblance to \'Dr. Freudstein\' from "House by the Cemetery"). The FX work is fairly good for such an obviously low-budget production (though I suspect most of it is kept in shadow for a reason), but overall, "Metamorphosis" leaves a bad retro aftertaste in your guts, in spite of its hopes to sway us otherwise. I can\'t help but agree

In [5]:
max_l = max([len(d['text']) for d in dataset["train"]])

In [6]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-5f446d4bfb444ffb.arrow
Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-36a0f29abbc0128a.arrow
Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-fd1075f44aed36a3.arrow
C:\Users\Junfe\AppData\Local\Temp/ipykernel_44680/1657904570.py:12: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  dataset.rename_column_("label", "labels")


In [7]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [8]:
# Add a new adapter
model.add_adapter("imdb")
# Add a matching classification head
model.add_classification_head(
    "imdb",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )



# Activate the adapter
model.train_adapter("imdb")

In [9]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [10]:
trainer.train()

***** Running training *****
  Num examples = 20000
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15000


Step,Training Loss
200,0.499500
400,0.244000
600,0.263700
800,0.241900
1000,0.224400
1200,0.231500
1400,0.196100
1600,0.205200
1800,0.191000
2000,0.228800


Saving model checkpoint to ./training_output\checkpoint-500
Configuration saved in ./training_output\checkpoint-500\imdb\adapter_config.json
Module weights saved in ./training_output\checkpoint-500\imdb\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-500\imdb\head_config.json
Module weights saved in ./training_output\checkpoint-500\imdb\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\imdb\head_config.json
Module weights saved in ./training_output\checkpoint-500\imdb\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\imdb\head_config.json
Module weights saved in ./training_output\checkpoint-500\imdb\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-1000
Configuration saved in ./training_output\checkpoint-1000\imdb\adapter_config.json
Module weights saved in ./training_output\checkpoint-1000\imdb\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-1000\imdb\

Module weights saved in ./training_output\checkpoint-6000\imdb\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-6000\imdb\head_config.json
Module weights saved in ./training_output\checkpoint-6000\imdb\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-6000\imdb\head_config.json
Module weights saved in ./training_output\checkpoint-6000\imdb\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-6500
Configuration saved in ./training_output\checkpoint-6500\imdb\adapter_config.json
Module weights saved in ./training_output\checkpoint-6500\imdb\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-6500\imdb\head_config.json
Module weights saved in ./training_output\checkpoint-6500\imdb\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-6500\imdb\head_config.json
Module weights saved in ./training_output\checkpoint-6500\imdb\pytorch_model_head.bin
Configuration saved in ./tra

Configuration saved in ./training_output\checkpoint-11500\imdb\head_config.json
Module weights saved in ./training_output\checkpoint-11500\imdb\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-12000
Configuration saved in ./training_output\checkpoint-12000\imdb\adapter_config.json
Module weights saved in ./training_output\checkpoint-12000\imdb\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-12000\imdb\head_config.json
Module weights saved in ./training_output\checkpoint-12000\imdb\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-12000\imdb\head_config.json
Module weights saved in ./training_output\checkpoint-12000\imdb\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-12000\imdb\head_config.json
Module weights saved in ./training_output\checkpoint-12000\imdb\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-12500
Configuration saved in ./training_output\

TrainOutput(global_step=15000, training_loss=0.1554879654566447, metrics={'train_runtime': 2108.956, 'train_samples_per_second': 56.9, 'train_steps_per_second': 7.113, 'total_flos': 3.212080128e+16, 'train_loss': 0.1554879654566447, 'epoch': 6.0})

In [11]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8


{'eval_loss': 0.2421850562095642,
 'eval_acc': 0.944,
 'eval_runtime': 39.2779,
 'eval_samples_per_second': 127.298,
 'eval_steps_per_second': 15.912,
 'epoch': 6.0}

In [12]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

[{'label': '👍', 'score': 0.9924499988555908}]

In [13]:
model.save_adapter("./final_adapter", "imdb")

!ls -lh final_adapter

Configuration saved in ./final_adapter\adapter_config.json
Module weights saved in ./final_adapter\pytorch_adapter.bin
Configuration saved in ./final_adapter\head_config.json
Module weights saved in ./final_adapter\pytorch_model_head.bin
'ls' is not recognized as an internal or external command,
operable program or batch file.
